In [ ]:
!nvidia-smi

In [ ]:
!wget https://github.com/onnx/models/raw/master/text/machine_comprehension/t5/model/t5-encoder-12.onnx

In [ ]:
mkdir -p ./models/t5/1

In [ ]:
configuration = """
name: "t5"
platform: "onnxruntime_onnx"
max_batch_size: 1
input [
 {
    name: "input_ids"
    data_type: TYPE_INT64
    dims: [ -1 ]
  }
]
output {
    name: "hidden_states"
    data_type: TYPE_FP32
    dims: [ -1, 768 ]
  }
"""

with open('./models/t5/config.pbtxt', 'w') as file:
    file.write(configuration)

In [ ]:
!cp ./t5-encoder-12.onnx ./models/t5/1/model.onnx

In [ ]:
!perf_analyzer \
  -m t5 \
  -x 1 \
  -b 1 \
  --concurrency-range 100 \
  -u triton:8000 \
  -i http \
  --input-data zero \
  --shape input_ids:20


In [ ]:
!cp -R ./models/t5 ./models/t5-cpu

In [ ]:
configuration = """
name: "t5-cpu"
platform: "onnxruntime_onnx"
max_batch_size: 1
input [
 {
    name: "input_ids"
    data_type: TYPE_INT64
    dims: [ -1 ]
  }
]
output {
    name: "hidden_states"
    data_type: TYPE_FP32
    dims: [ -1, 768 ]
  }
instance_group [
    {
      count: 1
      kind: KIND_CPU
    }
  ]
"""

with open('./models/t5-cpu/config.pbtxt', 'w') as file:
    file.write(configuration)

In [ ]:
!perf_analyzer \
  -m t5-cpu \
  -x 1 \
  -b 1 \
  --concurrency-range 100 \
  -u triton:8000 \
  -i http \
  --input-data zero \
  --shape input_ids:20


In [ ]:
!pip install onnx_opcounter

In [ ]:
from onnx_opcounter import calculate_params
import onnx

model = onnx.load_model('t5-encoder-12.onnx')
params = calculate_params(model)

print('Number of params:', params)